# Mimid :  Inferring Grammars from X86 Binaries

* Code for subjects [here](#Our-subject-programs)
* Evaluation starts [here](#Evaluation)
  * The evaluation on specific subjects starts [here](#Subjects)
    * [Calculator](#Calculator)
    * [Json](#Parson)
    * [MJS](#MJS)
    * [CSV](#CSV)
    * [TinyC](#TinyC)
    * [Mini-Lisp](#Komplott)
    * [Yxml](#Yxml)
    * [Duktape](#Duktape)
* Results are [here](#Results)

In [1]:
import fuzzingbook

## X86 instruction as an object

### Class Instruction

In [2]:
class Instruction:
    def __init__(self, instr):
        self.symbol_name = None
        self.pointed_address = None
        self.dest_reg = None
        self.instr_type = None
        self._parse(instr)

In [3]:
class Instruction(Instruction):
    def get_pointed_value(self, val):
        val = val.strip('%*')
        if val in REGISTERS:
            ptr_addr = gdb.execute('x/s $%s' % (val),
                                to_string=True).split(':')
            return ptr_addr[0]
        return val

In [4]:
class Instruction(Instruction):
    def resolve_addressing_mode(self, instr):
        str0 = instr.split(',')
        if len(str0) > 2:
            pass

        src = str0[-1]
        if '(' not in src:
            if src[1:] in REGISTERS:

                return '$%s' % src[1:]
        else:
            if src.startswith('-'):
                displacement, rest = tuple(src.split('(%'))
                return '$%s%s' % (rest[:-1], displacement)
            elif src.startswith('(') and src.endswith(')'):
                return '$%s' % src[2: -1]
            else:
                displacement, rest = tuple(src.split('(%'))
                return '$%s+%s' % (rest[:-1], displacement)

In [5]:
class Instruction(Instruction):
    def _parse(self, instr):
        instr_list = instr.split()
        instr_list.pop(0)

        self.current_address = instr_list[0]
        if "<" in instr_list[1]:
            instr_list.pop(1)
        self.instr_type = instr_list[1]

        if self.instr_type == CALL:
            self.pointed_address = self.get_pointed_value(instr_list[2])
            if len(instr_list) > 3:
                self.symbol_name = instr_list[-1]

        elif self.instr_type.startswith('mov') or self.instr_type == 'push' or \
            self.instr_type == 'pop':
            self.dest_reg = self.resolve_addressing_mode(instr_list[2])

### Extracting function names from binary

In [6]:
INP_ARR = []
VAL_TUPLE = []

In [7]:
def reset_helper():
    global INP_ARR 
    global VAL_TUPLE
    INP_ARR.clear()
    VAL_TUPLE.clear()

### Binary Miner

In [8]:
class BinaryDebugger:
    def __init__(self, inp, binary, fn_list):
        self.inp = inp
        self.binary = binary
        self.functions = fn_list
        self._set_logger()
        self.tree = {}
        self.mid = None
        self.method_map, self.m_stack = {}, []

In [9]:
class BinaryDebugger(BinaryDebugger):
    def break_at(self, address):
        gdb.execute("break *%s" % address)
    def finish(self):
        gdb.execute('finish')
    def get_instruction(self):
        return gdb.execute('x/i $rip', to_string=True)
    def nexti(self):
        gdb.execute('nexti')
    def resume(self):
        gdb.execute('continue')
    def run(self):
        gdb.execute('run')
    def step(self):
        gdb.execute('stepi')

In [10]:
class BinaryDebugger(BinaryDebugger):
    def start_program(self):
        gdb.execute("set args '%s'" % self.inp)
        gdb.execute("file %s" % self.binary)

In [11]:
class BinaryDebugger(BinaryDebugger):
    def _in_scope(self, instr, addr_range):
        s1, e1, s2, e2 = addr_range
        instr = instr.split()
        instr.pop(0)
        
        current_addr = instr[0].strip(':')
        hex_val = int(current_addr, 16)
        if hex_val in range(int(s1, 16), int(e1, 16)) or \
            hex_val in range(int(s2, 16), int(e2, 16)):
            return True
        else:
            return False

In [12]:
class BinaryDebugger(BinaryDebugger):
    def _get_entry_address(self):
        self.start_program()
        self.run()

        info_file = gdb.execute('info file', to_string=True)
        entry = None
        
        for line in info_file.splitlines():
            if 'Entry point' in line:
                entry = line.split(':')[1]
                break
        return entry

In [13]:
class BinaryDebugger(BinaryDebugger):
    def _set_logger(self):
        gdb.execute('set logging overwrite on')
        gdb.execute('set logging redirect on')
        gdb.execute('set logging on')

In [14]:
class BinaryDebugger(BinaryDebugger):
    def _get_address_range(self):
        s1 = s2 = None
        e1 = e2 = None
        mappings = gdb.execute('info proc mappings', to_string=True)

        for i, line in enumerate(mappings.splitlines()):
            if i == 4:
                s1 = line.split()[0]
            elif i == 6:
                e1 = line.split()[1]
            elif i == 7:
                s2 = line.split()[0]
            elif i == 10:
                e2 = line.split()[1]
        return (s1, e1, s2, e2)

In [15]:
class BinaryDebugger(BinaryDebugger):
    def _get_main_address(self):
        entry = self._get_entry_address()
        self.break_at(entry)
        gdb.execute('run')

        instr = []
        while True:
            next_i = self.get_instruction()
            if CALL in next_i:
                break
            instr.append(next_i)
            self.step()

        instr = instr[-1].split()
        if len(instr) == 6:
            s = instr[3]
        else:
            s = instr[4]

        reg = s[-3:]
        main_addr = gdb.execute('p/x $%s' % reg, to_string=True)
        main_addr = main_addr.partition("= ")
        main_addr = main_addr[-1]

        return main_addr

In [16]:
class BinaryDebugger(BinaryDebugger):
    def _lookup_address(self, addr, symbol):
        addr = addr.rstrip("\n")
        if addr in self.functions.keys():
            return self.functions[addr]
        else:
            if symbol:
                s0 = symbol[1:-1].split('@')[0]
                return s0
            return None

In [17]:
class BinaryDebugger(BinaryDebugger):
    def _init_methodMap_mtdStack(self, mname):
        self.method_map = {'0':[0, None, [1]], '1': [1, mname, []]}
        self.m_stack = ['0', '1']

In [18]:
class BinaryDebugger(BinaryDebugger):
    def _init_result(self, inp, arg1):
        self.result = {'inputstr': inp,
                'arg': inp,
                'original': arg1,
                'comparisons': []}

In [19]:
class BinaryDebugger(BinaryDebugger):
    def event_loop(self):
        main = self._get_main_address()
        mname = self._lookup_address(main, None)
        cs = CallStack()
        cs.enter(mname)

        self._init_methodMap_mtdStack(mname)     
        self._init_result(self.inp, arg1)  
        self.break_at(main)
        self.resume()
        addr_range = self._get_address_range()

        while True:
            try:
                nexti = self.get_instruction()
                if self._in_scope(nexti, addr_range):
                    h = Instruction(nexti)
                    if h.instr_type == CALL:
                        name = self._lookup_address(h.pointed_address, h.symbol_name)
                        if not name:
                            self.step()
                            self.finish()
                        else:
                            self.step()
                            cs.enter(name)
                            x, self.mid = cs.method_id
                            self.method_map[self.m_stack[-1]][-1].append(self.mid)
                            self.method_map[str(self.mid)] = [self.mid, name, []]
                            self.m_stack.append(str(self.mid))
                    elif h.instr_type == RETURN:
                        self.step()
                        cs.leave()
                        if len(self.m_stack) > 1: self.m_stack.pop()
                    else:
                        self.step()
                        val = read_register_val(h.dest_reg, self.inp)
                        comparison = process_value(val, self.mid, self.inp)
                        if comparison != None:
                            self.result['comparisons'].append(comparison)
                else:
                    self.finish()
            except gdb.error:
                break
        self.result['method_map'] = self.method_map
        with open('tree', 'w+') as f:
            obj = jsonpickle.encode(self.result)
            f.write(obj)

In [20]:
from fuzzingbook import fuzzingbook_utils
import sys

In [21]:
head = """\
import sys
sys.path.extend([%s])
sys.path.append('.')
import matplotlib.pyplot
matplotlib.pyplot._IP_REGISTERED = True # Hack
#import fuzzingbook_utils
import fuzzingbook
from fuzzingbook.GrammarMiner import CallStack
import jsonpickle
import os, subprocess
import gdb
import re, json
""" % (', '.join("'%s'" % str(i) for i in sys.path if i))
Instruction_src = fuzzingbook_utils.extract_class_definition(Instruction)
BinaryDebugger_src = fuzzingbook_utils.extract_class_definition(BinaryDebugger)

declaration = """
CALL = 'callq'
RETURN = 'retq'
LINE = 'line'
ARG_REGISTERS = ['rdi', 'rsi', 'rdx', 'rcx', 'r8', 'r9', 'edi', 'rbx']
REGISTERS = ARG_REGISTERS + ['rax', 'eax', 'edi' 'esi', 'edx', 'ecx', 'rsp', 'rbp']
"""
helper1 = """
INP_ARR = []
VAL_TUPLE = []

def reset_helper():
    global INP_ARR 
    global VAL_TUPLE
    INP_ARR.clear()
    VAL_TUPLE.clear()
    
def get_names_from_symbols(objfile):
    names = []
    for name in objfile:
        name = name.split()
        name = name[-1].decode('utf-8')
        if '@@' in name:
            names.append(name.split('@@')[0])
            continue
        names.append(name)
    return names

def list_objfile_symbols():
    proc = subprocess.Popen(['nm', 'a.out'], stdout=subprocess.PIPE)
    output = proc.stdout.read()
    output = output.splitlines()
    return output

def get_function_names(inp, binary):
    fn_dict = {}
    fn_names = []

    symbols = list_objfile_symbols()
    functions = get_names_from_symbols(symbols)

    gdb.execute("set args '%s'" % inp)
    gdb.execute("file %s" % binary)
    gdb.execute('set confirm off')
    gdb.execute('run')
    for k in functions:
        try:
            s = gdb.execute('info address %s' % k,
            to_string=True).split(' ')
            if s[4].startswith('0x'):
                v = s[4].rstrip()
                u = v.strip('.')
                fn_dict[v] = k
            else:
                u = s[-1].rstrip()
                u = u.strip('.')
                fn_dict[u] = k
        except gdb.error:
            continue
    return fn_dict
"""
helper2 = """
def read_register_val(reg, original):
    if not reg:
        return None

    val = read_as_string(reg)
    if not val:
        return None
    if val == '""':
        a = read_ptr_addr(reg)
        return read_register_val(a, original)
    elif val in original:
        return val
    else:
        x = val[1: -1] if val[0] == '"' and val[-1] == '"' else val
        return x if x in original else None

def read_ptr_addr(reg):
    try:
        str1 = gdb.execute('x/a %s' % (reg), to_string=True)
        for idx, char in enumerate(str1):
            if str1[idx] == ':':
                addr_val = str1[idx + 1:]
                addr_val = addr_val.strip()
                break
        return addr_val
    except Exception:
        return
        
def read_as_string(reg):
    try:
        str0 = gdb.execute('x/s %s' % (reg), to_string=True)
        if '<error:' in str0 and not reg.startswith('0x'):
            x = gdb.execute('p/c %s' % reg, to_string=True)
            x = x.split()
            x = x[-1]
            return x[1:-1]

        for idx, char in enumerate(str0):
            if str0[idx] == ':':
                str_val = str0[idx + 1:]
                str_val = str_val.strip()
                break
        return str_val
    except Exception:
        return

def process_value(val, mid, inputstr):
    if val and len(val) == 1:
        INP_ARR.append(val)
        x = ''.join(INP_ARR)

        if inputstr.startswith(x) and mid not in VAL_TUPLE:
            VAL_TUPLE.append(mid)
            idx = len(INP_ARR) - 1
            return [idx, val, mid]
        else:
            INP_ARR.pop()
"""
tail = """
reset_helper()
arg_0 = None
with open(f'inp.0.txt', 'r+') as f:
    arg_0 = f.read().strip()

fnames = get_function_names(arg_0, "a.out")
subprocess.call(['strip', '-s', "a.out"])

debugger = BinaryDebugger(arg_0, 'a.out', fnames)
debugger.event_loop()
"""
miner_src = '\n'.join([head, declaration, Instruction_src, helper1, BinaryDebugger_src, helper2, tail])

In [22]:
with open('bminer.py', 'w+') as f:
    print(miner_src, file=f)

### Reconstructing the Method Tree with Attached Character Comparisons

Reconstruct the actual method trace from a trace with the following
format
```
key   : [ mid, method_name, children_ids ]
```

In [23]:
def reconstruct_method_tree(method_map):
    first_id = None
    tree_map = {}
    for key in method_map:
        m_id, m_name, m_children = method_map[key]
        children = []
        if m_id in tree_map:
            # just update the name and children
            assert not tree_map[m_id]
            tree_map[m_id]['id'] = m_id
            tree_map[m_id]['name'] = m_name
            tree_map[m_id]['indexes'] = []
            tree_map[m_id]['children'] = children
        else:
            assert first_id is None
            tree_map[m_id] = {'id': m_id, 'name': m_name, 'children': children, 'indexes': []}
            first_id = m_id

        for c in m_children:
            assert c not in tree_map
            val = {}
            tree_map[c] = val
            children.append(val)
    return first_id, tree_map

In [24]:
from fuzzingbook.GrammarFuzzer import display_tree

#### Identifying last comparisons
We need only the last comparisons made on any index. This means that we should care for only the last parse in an ambiguous parse. So, we assign the method that last touched an index to be its consumer.

However, to make concessions for real world, we also check if we are overwriting a child (`HEURISTIC`). Essentially, if the heursitic is enabled, then if the current method id (`midP`) is smaller than the `midC` already stored in the last comparison map, then it means that `midP` is a parent that called `midC` previously, and now accessing an index that `midC` touched. This happens when the parent tries to find a substring like `#` in the entirety of the original string. (Note that we have seen this only in `URLParser`). (Note that this heuristic does not restrict reparsing by another function call -- in such a case, `midC` will not smaller than `midP`). So, perhaps, we should let the child keep the ownership. However, there is one more wrinkle. If the character being contested was the last index touched by our `mid`, then it is likely that it was simply a boundary check. In that case, we should let the parent own this character.

In [25]:
LAST_COMPARISON_HEURISTIC = False

In [26]:
def last_comparisons(comparisons):
    last_cmp_only = {}
    last_idx = {}

    # get the last indexes compared in methods.
    for idx, char, mid in comparisons:
        if mid in last_idx:
            if idx > last_idx[mid]:
                last_idx[mid] = idx
        else:
            last_idx[mid] = idx

    for idx, char, mid in comparisons:
        if LAST_COMPARISON_HEURISTIC:
            if idx in last_cmp_only:
                midC = last_cmp_only[idx]
                if midC > mid:
                    # midC is a child of mid.
                    # do not clobber children unless it was the last character
                    # for that child.
                    if last_idx[mid] == idx:
                        # if it was the last index, may be the child used it
                        # as a boundary check.
                        pass
                    else:
                        # do not overwrite the current value of `last_cmp_only[idx]`
                        continue
        last_cmp_only[idx] = mid
    return last_cmp_only

#### Attaching characters to the tree
Add the comparison indexes to the method tree that we constructed

In [27]:
def attach_comparisons(method_tree, comparisons):
    for idx in comparisons:
        mid = comparisons[idx]
        method_tree[mid]['indexes'].append(idx)

Convert a list of indexes to a corresponding terminal tree node

In [28]:
def to_node(idxes, my_str):
    assert len(idxes) == idxes[-1] - idxes[0] + 1
    assert min(idxes) == idxes[0]
    assert max(idxes) == idxes[-1]
    return my_str[idxes[0]:idxes[-1] + 1], [], idxes[0], idxes[-1]

In [29]:
from operator import itemgetter
import itertools as it

We now need to identify the terminal (leaf) nodes. For that, we want to group contiguous letters in a node together, and call it a leaf node. So, convert our list of indexes to lists of contiguous indexes first, then convert them to terminal tree nodes. Then, return a set of one level child nodes with contiguous chars from indexes.

In [30]:
def indexes_to_children(indexes, my_str):
    lst = [
        list(map(itemgetter(1), g))
        for k, g in it.groupby(enumerate(indexes), lambda x: x[0] - x[1])
    ]

    return [to_node(n, my_str) for n in lst]

Finally, we need to remove the overlap from the trees we have so far. The idea is that, given a node, each child node of that node should be uniquely responsible for a specified range of characters, with no overlap allowed between the children. The starting of the first child to ending of the last child will be the range of the node.

#### Removing Overlap
If overlap is found, the tie is biased to the later child. That is, the later child gets to keep the range, and the former child is recursively traversed to remove overlaps from its children. If a child is completely included in the overlap, the child is excised. A few convenience functions first:

In [31]:
def does_item_overlap(r, r_):
    (s, e), (s_, e_) = r, r_
    return ((s_ >= s and s_ <= e) or 
            (e_ >= s and e_ <= e) or 
            (s_ <= s and e_ >= e))

In [32]:
def is_second_item_included(r, r_):
    (s, e), (s_, e_) = r, r_
    return (s_ >= s and e_ <= e)

In [33]:
def has_overlap(ranges, r_):
    return {r for r in ranges if does_item_overlap(r, r_)}

In [34]:
def is_included(ranges, r_):
    return {r for r in ranges if is_second_item_included(r, r_)}

In [35]:
def remove_overlap_from(original_node, orange):
    node, children, start, end = original_node
    new_children = []
    if not children:
        return None
    start = -1
    end = -1
    for child in children:
        if does_item_overlap(child[2:4], orange):
            new_child = remove_overlap_from(child, orange)
            if new_child: # and new_child[1]:
                if start == -1: start = new_child[2]
                new_children.append(new_child)
                end = new_child[3]
        else:
            new_children.append(child)
            if start == -1: start = child[2]
            end = child[3]
    if not new_children:
        return None
    assert start != -1
    assert end != -1
    return (node, new_children, start, end)

Verify that there is no overlap.

In [36]:
def no_overlap(arr):
    my_ranges = {}
    for a in arr:
        _, _, s, e = a
        r = (s, e)
        included = is_included(my_ranges, r)
        if included:
            continue  # we will fill up the blanks later.
        else:
            overlaps = has_overlap(my_ranges, r) 
            if overlaps:
                # unlike include which can happen only once in a set of
                # non-overlapping ranges, overlaps can happen on multiple parts.
                # The rule is, the later child gets the say. So, we recursively
                # remove any ranges that overlap with the current one from the
                # overlapped range.
                assert len(overlaps) == 1
                oitem = list(overlaps)[0]
                v = remove_overlap_from(my_ranges[oitem], r)
                del my_ranges[oitem]
                if v:
                    my_ranges[v[2:4]] = v
                my_ranges[r] = a
            else:
                my_ranges[r] = a
    res = my_ranges.values()
    # assert no overlap, and order by starting index
    s = sorted(res, key=lambda x: x[2])
    return s

#### Generate derivation tree

Convert a mapped tree to the _fuzzingbook_ style derivation tree.

In [37]:
def to_tree(node, my_str):
    method_name = ("<%s>" % node['name']) if node['name'] is not None else '<START>'
    indexes = node['indexes']
    node_children = [to_tree(c, my_str) for c in node.get('children', [])]
    idx_children = indexes_to_children(indexes, my_str)
    children = no_overlap([c for c in node_children if c is not None] + idx_children)
    if not children:
        return None
    start_idx = children[0][2]
    end_idx = children[-1][3]
    si = start_idx
    my_children = []
    # FILL IN chars that we did not compare. This is likely due to an i + n
    # instruction.
    for c in children:
        if c[2] != si:
            sbs = my_str[si: c[2]]
            my_children.append((sbs, [], si, c[2] - 1))
        my_children.append(c)
        si = c[3] + 1

    m = (method_name, my_children, start_idx, end_idx)
    return m

In [38]:
from IPython.display import Image

In [39]:
def zoom(v, zoom=True):
    # return v directly if you do not want to zoom out.
    if zoom:
        return Image(v.render(format='png'))
    return v

## Generating a Grammar

Generating a grammar from the generalized derivation trees is pretty simple. Start at the start node, and any node that represents a method or a pseudo method becomes a nonterminal. The children forms alternate expansions for the nonterminal. Since all the keys are compatible, merging the grammar is simply merging the hash map.

First, we define a pretty printer for grammar.

In [40]:
import re
RE_NONTERMINAL = re.compile(r'(<[^<> ]*>)')

In [41]:
def recurse_grammar(grammar, key, order, canonical):
    rules = sorted(grammar[key])
    old_len = len(order)
    for rule in rules:
        if not canonical:
            res =  re.findall(RE_NONTERMINAL, rule)
        else:
            res = rule
        for token in res:
            if token.startswith('<') and token.endswith('>'):
                if token not in order:
                    order.append(token)
    new = order[old_len:]
    for ckey in new:
        recurse_grammar(grammar, ckey, order, canonical)

In [42]:
def show_grammar(grammar, start_symbol='<START>', canonical=True):
    order = [start_symbol]
    recurse_grammar(grammar, start_symbol, order, canonical)
    if len(order) != len(grammar.keys()):
        assert len(order) < len(grammar.keys())
    return {k: sorted(grammar[k]) for k in order}

### Trees to grammar

In [43]:
def to_grammar(tree, grammar):
    node, children, _, _ = tree
    if not children: return grammar
    tokens = []
    if node not in grammar:
        grammar[node] = list()
    for c in children:
        tokens.append(c[0])
        to_grammar(c, grammar)
    grammar[node].append(tuple(tokens))
    return grammar

In [44]:
def merge_grammar(g1, g2):
    all_keys = set(list(g1.keys()) + list(g2.keys()))
    merged = {}
    for k in all_keys:
        alts = set(g1.get(k, []) + g2.get(k, []))
        merged[k] = alts
    return {k:[l for l in merged[k]] for k in merged}

In [45]:
def convert_to_grammar(my_trees):
    grammar = {}
    ret = []
    for my_tree in my_trees:
        tree = my_tree['tree']
        start = tree[0]
        src_file = my_tree['original']
        arg_file = my_tree['arg']
        ret.append((start, src_file, arg_file))
        g = to_grammar(tree, grammar)
        grammar = merge_grammar(grammar, g)
    return grammar

### Inserting Empty Alternatives for IF and Loops

Next, we want to insert empty rules for those loops and conditionals that can be skipped. For loops, the entire sequence has to contain the empty marker.

In [46]:
def check_empty_rules(grammar):
    new_grammar = {}
    for k in grammar:
        if k in ':if_':
            name, marker = k.split('#')
            if name.endswith(' *'):
                new_grammar[k] = grammar[k].add(('',))
            else:
                new_grammar[k] = grammar[k]
        elif k in ':while_':
            # TODO -- we have to check the rules for sequences of whiles.
            # for now, ignore.
            new_grammar[k] = grammar[k]
        else:
            new_grammar[k] = grammar[k]
    return new_grammar

### Learning Regular Expressions

We now need to generalize the loops. The idea is to look for patterns exclusively in the similarly named while loops using any of the regular expression learners. For the prototype, we replaced the modified Sequitur with the modified Fernau which gave us better regular expressions than before. The main constraint we have is that we want to avoid repeated execution of program if possible. Fernau algorithm can recover a reasonably approximate regular exression based only on positive data.

#### The modified Fernau algorithm

The Fernau algorithm is from _Algorithms for learning regular expressions from positive data_ by _HenningFernau_. Our algorithm uses a modified form of the Prefix-Tree-Acceptor from Fernau. First we define an LRF buffer of a given size.

In [47]:
class Buf:
    def __init__(self, size):
        self.size = size
        self.items = [None] * self.size

The `add1()` takes in an array, and transfers the first element of the array into the end of current buffer, and simultaneously drops the first element of the buffer.

In [48]:
class Buf(Buf):
    def add1(self, items):
        self.items.append(items.pop(0))
        return self.items.pop(0)

For equality between the buffer and an array, we only compare when both the array and the items are actually elements and not chunked arrays.

In [49]:
class Buf(Buf):
    def __eq__(self, items):
        if any(isinstance(i, dict) for i in self.items): return False
        if any(isinstance(i, dict) for i in items): return False
        return items == self.items

The `detect_chunks()` detects any repeating portions of a list of `n` size.

In [50]:
def detect_chunks(n, lst_):
    lst = list(lst_)
    chunks = set()
    last = Buf(n)
    # check if the next_n elements are repeated.
    for _ in range(len(lst) - n):
        lnext_n = lst[0:n]
        if last == lnext_n:
            # found a repetition.
            chunks.add(tuple(last.items))
        else:
            pass
        last.add1(lst)
    return chunks

Once we have detected plausible repeating sequences, we gather all similar sequences into arrays.

In [51]:
def chunkify(lst_,n , chunks):
    lst = list(lst_)
    chunked_lst = []
    while len(lst) >= n:
        lnext_n = lst[0:n]
        if (not any(isinstance(i, dict) for i in lnext_n)) and tuple(lnext_n) in chunks:
            chunked_lst.append({'_':lnext_n})
            lst = lst[n:]
        else:
            chunked_lst.append(lst.pop(0))
    chunked_lst.extend(lst)
    return chunked_lst

The `identify_chunks()` simply calls the `detect_chunks()` on all given lists, and then converts all chunks identified into arrays.

In [52]:
def identify_chunks(my_lsts):
    # initialize
    all_chunks = {}
    maximum = max(len(lst) for lst in my_lsts)
    for i in range(1, maximum//2+1):
        all_chunks[i] = set()

    # First, identify chunks in each list.
    for lst in my_lsts:
        for i in range(1,maximum//2+1):
            chunks = detect_chunks(i, lst)
            all_chunks[i] |= chunks

    # Then, chunkify
    new_lsts = []
    for lst in my_lsts:
        for i in range(1,maximum//2+1):
            chunks = all_chunks[i]
            lst = chunkify(lst, i, chunks)
        new_lsts.append(lst)
    return new_lsts

##### Prefix tree acceptor

The prefix tree acceptor is a way to represent positive data. The `Node` class holds a single node in the prefix tree acceptor.

In [53]:
class Node:
    # Each tree node gets its unique id.
    _uid = 0
    def __init__(self, item):
        # self.repeats = False
        self.count = 1 # how many repetitions.
        self.counters = set()
        self.last = False
        self.children = []
        self.item = item
        self.uid = Node._uid
        Node._uid += 1

    def update_counters(self):
        self.counters.add(self.count)
        self.count = 0
        for c in self.children:
            c.update_counters()

    def __repr__(self):
        return str(self.to_json())

    def __str__(self):
        return str("(%s, [%s])", (self.item, ' '.join([str(i) for i in self.children])))

    def to_json(self):
        s = ("(%s)" % ' '.join(self.item['_'])) if isinstance(self.item, dict) else str(self.item)
        return (s, tuple(self.counters), [i.to_json() for i in self.children])

    def inc_count(self):
        self.count += 1

    def add_ref(self):
        self.count = 1

    def get_child(self, c):
        for i in self.children:
            if i.item == c: return i
        return None

    def add_child(self, c):
        # first check if it is the current node. If it is, increment
        # count, and return ourselves.
        if c == self.item:
            self.inc_count()
            return self
        else:
            # check if it is one of the children. If it is a child, then
            # preserve its original count.
            nc = self.get_child(c)
            if nc is None:
                nc = Node(c)
                self.children.append(nc)
            else:
                nc.add_ref()
            return nc

The `update_tree()` essentially transforms a list of nodes to a chain of nodes starting at `root` if the `root` is an empty tree. If the `root` already contains a tree, the `update_tree()` traverses the path represented by `lst_` and makes a new child branch where the path specified doesn't exist in the tree.

In [54]:
def update_tree(lst_, root):
    lst = list(lst_)
    branch = root
    while lst:
        first, *lst = lst
        branch = branch.add_child(first)
    branch.last = True
    return root

Given a number of lists, the `create_tree_with_lists()` creates an actual tree out of these lists.

In [55]:
def create_tree_with_lsts(lsts):
    Node._uid = 0
    root =  Node(None)
    for lst in lsts:
        root.count = 1 # there is at least one element.
        update_tree(lst, root)
        root.update_counters()
    return root

Given a node, and a key, return the key and alts as a dict.

In [56]:
def get_star(node, key):
    if node.item is None:
        return [], {}
    if isinstance(node.item, dict):
        # take care of counters
        elements = node.item['_']
        my_key = "<%s-%d-s>" % (key, node.uid)
        alts = [elements]
        if len(node.counters) > 1: # repetition
            alts.append(elements + [my_key])
        return [my_key], {my_key:alts}
    else:
        return [str(node.item)], {}

In [57]:
def node_to_grammar(node, grammar, key):
    rule = []
    alts = [rule]
    if node.uid == 0:
        my_key = "<%s>" % key
    else:
        my_key = "<%s-%d>" % (key, node.uid)
    grammar[my_key] = alts
    if node.item is not None:
        mk, g = get_star(node, key)
        rule.extend(mk)
        grammar.update(g)
    # is the node last?
    if node.last:
        assert node.item is not None
        # add a duplicate rule that ends here.
        ending_rule = list(rule)
        # if there are no children, the current rule is
        # any way ending.
        if node.children:
            alts.append(ending_rule)

    if node.children:
        if len(node.children) > 1:
            my_ckey = "<%s-%d-c>" % (key, node.uid)
            rule.append(my_ckey)
            grammar[my_ckey] = [ ["<%s-%d>" % (key, c.uid)] for c in node.children]
        else:
            my_ckey = "<%s-%d>" % (key, node.children[0].uid)
            rule.append(my_ckey)
    else:
        pass
    for c in node.children:
        node_to_grammar(c, grammar, key)
    return grammar

def generate_grammar(lists, key):
    lsts = identify_chunks(lists)
    tree = create_tree_with_lsts(lsts)
    grammar = {}
    node_to_grammar(tree, grammar, key)
    return grammar

Given a rule, determine the abstraction for it.

In [58]:
def collapse_alts(rules, k):
    ss = [[str(r) for r in rule] for rule in rules]
    x = generate_grammar(ss, k[1:-1])
    return x

In [59]:
def collapse_rules(grammar):
    r_grammar = {}
    for k in grammar:
        new_grammar = collapse_alts(grammar[k], k)
        # merge the new_grammar with r_grammar
        # we know none of the keys exist in r_grammar because
        # new keys are k prefixed.
        for k_ in new_grammar:
            r_grammar[k_] = new_grammar[k_]
    return r_grammar

In [60]:
def convert_spaces_in_keys(grammar):
    keys = {key: key.replace(' ', '_') for key in grammar}
    new_grammar = {}
    for key in grammar:
        new_alt = []
        for rule in grammar[key]:
            new_rule = []
            for t in rule:
                for k in keys:
                    t = t.replace(k, keys[k])
                new_rule.append(t)
            new_alt.append(new_rule)
        new_grammar[keys[key]] = new_alt
    return new_grammar

### Remove duplicate and redundant entries

**IMPORTANT** we indicate things that operate on canonical by _c, and those that operate on fuzzable by _f, and both by _cf

In [61]:
def first_in_chain(token, chain):
    while True:
        if token in chain:
            token = chain[token]
            assert isinstance(token, str)
        else:
            break
    return token

Return a new symbol for `grammar` based on `symbol_name`.

In [62]:
def new_symbol(grammar, symbol_name="<symbol>"):
    if symbol_name not in grammar:
        return symbol_name

    count = 1
    while True:
        tentative_symbol_name = symbol_name[:-1] + "-" + repr(count) + ">"
        if tentative_symbol_name not in grammar:
            return tentative_symbol_name
        count += 1

Replace keys that have a single token definition with the token in the defition.

In [63]:
def replacement_candidate_chains(grammar, ignores):
    to_replace = {}
    for k in grammar:
        if k in ignores: continue
        if len(grammar[k]) != 1: continue
        rule = grammar[k][0]
        if len(rule) != 1: continue
        if is_nt(rule[0]):
            to_replace[k] = rule[0]
        else:
            pass
    return to_replace

In [64]:
def replace_key_by_new_key(grammar, keys_to_replace):
    new_grammar = {}
    for key in grammar:
        new_rules = []
        for rule in grammar[key]:
            new_rule = [keys_to_replace.get(token, token)
                        for token in rule]
            new_rules.append(new_rule)
        new_grammar[keys_to_replace.get(key, key)] = new_rules
    assert len(grammar) == len(new_grammar)
    return new_grammar

In [65]:
def replace_key_by_key(grammar, keys_to_replace):
    new_grammar = {}
    for key in grammar:
        if key in keys_to_replace: continue
        new_rules = []
        for rule in grammar[key]:
            for t in rule:
                assert isinstance(t, str)
            new_rule = [first_in_chain(token, keys_to_replace) for token in rule]
            new_rules.append(new_rule)
        new_grammar[key] = new_rules
    return new_grammar

In [66]:
def remove_single_entries(grammar):
    keys_to_replace = replacement_candidate_chains(grammar, {start_symbol, '<main>'})
    return replace_key_by_key(grammar, keys_to_replace)

Remove keys that have similar rules.

In [67]:
def collect_duplicate_rule_keys(grammar):
    collect = {}
    for k in grammar:
        salt = str(sorted(grammar[k]))
        if salt not in collect:
            collect[salt] = (k, set())
        else:
            collect[salt][1].add(k)
    return collect

In [68]:
def remove_duplicate_rule_keys(grammar):
    g = grammar
    while True:
        collect = collect_duplicate_rule_keys(g)
        keys_to_replace = {}
        for salt in collect:
            k, st = collect[salt]
            for s in st:
                keys_to_replace[s] = k
        if not keys_to_replace:
            break
        g = replace_key_by_key(g, keys_to_replace)
    return g

Remove all the control flow vestiges from names, and simply name them sequentially.

In [69]:
def deep_copy(t): # Python deepcopy is a bit buggy
    v = json.dumps(t)
    return json.loads(v)

In [70]:
def collect_replacement_keys(grammar):
    g = deep_copy(grammar)
    to_replace = {}
    for k in grammar:
        if ':' in k:
            first, rest = k.split(':')
            sym = new_symbol(g, symbol_name=first + '>')
            assert sym not in g
            g[sym] = None
            to_replace[k] = sym
        else:
            continue
    return to_replace

Remove keys that are referred to only from a single rule, and which have a single alternative.
Import. This can't work on canonical representation. First, given a key, we figure out its distance to `<START>`.

This is different from `remove_single_entries()` in that, there we do not care if the key is being used multiple times. Here, we only replace keys that are referred to only once.

In [71]:
import math

In [72]:
def len_to_start(item, parents, start_symbol, seen=None):
    if seen is None: seen = set()
    if item in seen: return math.inf
    seen.add(item)
    if item == start_symbol: return 0
    else: return 1 + min(len_to_start(p, parents, start_symbol, seen)
                         for p in parents[item])

In [73]:
def order_by_length_to_start(items, parent_map, start_symbol):
    return sorted(items, key=lambda i: len_to_start(i, parent_map, start_symbol))

Next, we generate a map of `child -> [parents]`.

In [74]:
def get_parents_of_tokens(grammar, key, seen=None, parents=None):
    if parents is None: parents, seen = {}, set()
    if key in seen: return parents
    seen.add(key)
    for res in grammar[key]:
        for token in res:
            if not is_nt(token): continue
            parents.setdefault(token, []).append(key)
    for ckey in {i for i in  grammar if i not in seen}:
        get_parents_of_tokens(grammar, ckey, seen, parents)
    return parents

In [75]:
def remove_references(keys_to_replace):
    to_process = list(keys_to_replace.keys())
    updated_dict = {}
    references = {}
    order = []
    while to_process:
        key, *to_process = to_process
        rule = keys_to_replace[key]
        new_rule = []
        skip = False
        for token in rule:
            if token not in updated_dict:
                if token in to_process:
                    # so this token will get defined later. We simply postpone
                    # the processing of this key until that key is defined.
                    # TODO: check for cycles.
                    to_process.append(key)
                    references.setdefault(token, set()).add(key)
                    skip = True
                    break
                else:
                    new_rule.append(token)
            else:
                new_rule.extend(updated_dict[token])
        if not skip:
            order.append(key)
            updated_dict[key] = new_rule
    return updated_dict

In [76]:
def replace_keys_by_rule(grammar, keys_to_replace):
    # we now need to verify that none of the keys are part of the sequences.
    keys_to_replace = remove_references(keys_to_replace)

    new_grammar = {}
    for key in grammar:
        if key in keys_to_replace: continue

        new_rules = []
        for rule in grammar[key]:
            new_rule = []
            for token in rule:
                if token in keys_to_replace:
                    new_rule.extend(keys_to_replace[token])
                else:
                    new_rule.append(token)
            new_rules.append(new_rule)
        new_grammar[key] = new_rules
    return new_grammar

Now, all together.

In [77]:
def remove_single_alts(grammar, start_symbol):
    single_alts = {p for p in grammar if len(grammar[p]) == 1 and p != start_symbol}

    child_parent_map = get_parents_of_tokens(grammar, start_symbol)
    assert len(child_parent_map) < len(grammar)

    single_refs = {p:child_parent_map[p] for p in single_alts if len(child_parent_map[p]) <= 1}

    ordered = order_by_length_to_start(single_refs, child_parent_map, start_symbol)

    for p in ordered:
        assert len(grammar[p]) == 1
        if not isinstance(grammar[p][0], str):
            print(p, grammar[p][0])

    keys_to_replace = {p:grammar[p][0] for p in ordered}
    g =  replace_keys_by_rule(grammar, keys_to_replace)
    return g

remove similar rules from under a single key

In [78]:
def len_rule(r): return len(r)
def len_definition(d): return sum([len_rule(r) for r in d])
def len_grammar(g): return sum([len_definition(g[k]) for k in g])

In [79]:
def remove_duplicate_rules_in_a_key(g):
    g_ = {}
    for k in g:
        s = {str(r):r for r in g[k]}
        g_[k] = list(sorted(list(s.values())))
    return g_

In [80]:
def grammar_gc(grammar, start_symbol):
    def strip_key(grammar, key, order):
        rules = sorted(grammar[key])
        old_len = len(order)
        for rule in rules:
            for token in rule:
                if is_nt(token):
                    if token not in order:
                        order.append(token)
        new = order[old_len:]
        for ckey in new:
            strip_key(grammar, ckey, order)

    order = [start_symbol]
    strip_key(grammar, start_symbol, order)
    assert len(order) == len(grammar.keys())
    g = {k: sorted(grammar[k]) for k in order}
    for k in g:
        for r in g[k]:
            for t in r:
                assert isinstance(t, str)
    return g

In [81]:
def cleanup_grammar(g, start_symbol):
    g = grammar_gc(g, start_symbol)
    g1 = check_empty_rules(g) # add optional rules
    g1 = grammar_gc(g1, start_symbol)
 
    g2 = collapse_rules(g1) # learn regex
    g2 = grammar_gc(g2, start_symbol)

    g3 = convert_spaces_in_keys(g2) # fuzzable grammar
    g3 = grammar_gc(g3, start_symbol)
    return g3

In [82]:
def remove_single_entry_chains(grammar, start_symbol):
    keys_to_replace = replacement_candidate_chains(grammar, {start_symbol, '<main>'})
    return replace_key_by_key(grammar, keys_to_replace)

In [83]:
def cleanup_token_names(grammar):
    keys_to_replace = collect_replacement_keys(grammar)
    g = replace_key_by_new_key(grammar, keys_to_replace)
    return g

In [84]:
def remove_self_definitions(g):
    g_ = {}
    for k in g:
        rs_ = []
        for r in g[k]:
            assert not isinstance(r, str)
            if len(r) == 1 and r[0] == k: continue
            rs_.append(r)
        g_[k] = rs_
    return g_

In [85]:
def compact_grammar(e, start_symbol):
    assert start_symbol in e
    l = len_grammar(e)
    diff = 1
    while diff > 0:
        assert start_symbol in e
        e = remove_single_entry_chains(e, start_symbol)
        e = grammar_gc(e, start_symbol) # garbage collect

        e = remove_duplicate_rule_keys(e)
        e = grammar_gc(e, start_symbol) # garbage collect

        e = cleanup_token_names(e)
        e = grammar_gc(e, start_symbol) # garbage collect

        e = remove_single_alts(e, start_symbol)
        e = grammar_gc(e, start_symbol) # garbage collect

        e = remove_duplicate_rules_in_a_key(e)
        e = grammar_gc(e, start_symbol) # garbage collect

        e = remove_self_definitions(e)
        e = grammar_gc(e, start_symbol) # garbage collect

        l_ = len_grammar(e)
        diff = l - l_
        l = l_
    e = grammar_gc(e, start_symbol)
    return e

### The Complete Miner

We now put everything together. The `miner()` takes the traces, produces trees out of them, and verifies that the trees actually correspond to the input.

In [86]:
def is_nt(v):
    return len(v) > 1 and (v[0], v[-1]) == ('<', '>')

In [87]:
def tree_to_str(tree): # Non recursive
    expanded = []
    to_expand = [tree]
    while to_expand:
        (key, children, *rest), *to_expand = to_expand
        if is_nt(key):
            to_expand = children + to_expand
        else:
            assert not children
            expanded.append(key)
    return ''.join(expanded)

In [88]:
def miner(call_traces):
    my_trees = []
    for call_trace in call_traces:
        method_map = call_trace['method_map']

        first, method_tree = reconstruct_method_tree(method_map)
        comparisons = call_trace['comparisons']
        attach_comparisons(method_tree, last_comparisons(comparisons))

        my_str = call_trace['inputstr']

        tree = to_tree(method_tree[first], my_str)
        my_tree = {'tree': tree, 'original': call_trace['original'], 'arg': call_trace['arg']}
        print(tree_to_str(tree))
        assert tree_to_str(tree) == my_str
        my_trees.append(my_tree)
    return my_trees

In [89]:
import jsonpickle
from fuzzingbook.Parser import non_canonical, canonical
def binary_miner(seeds, executable):
    call_trace = []
    for inp in seeds:
        arg0 = '\'py arg0="%s"\'' % inp
        arg1 = '\'py arg1="%s"\'' % executable
        
        with open(f'inp.0.txt', 'w+') as f:
            print(inp, file=f)
        
        !gcc -g {executable}
        !gdb --batch-silent -ex {arg1} -x bminer.py
        
        with open(f'tree', 'rb') as f:
            call_trace.append((jsonpickle.decode(f.read())))
            
    mined_tree = miner(call_trace)
    g0 = convert_to_grammar(mined_tree)
    g = cleanup_grammar(g0, start_symbol='<START>')
    g = compact_grammar(g, start_symbol='<START>')
    return show_grammar(non_canonical(g), canonical=False)

# Evaluation

In [90]:
Max_Precision = 1000
Max_Recall = 1000
BMiner = {}
BMinerFuzz = {}
BMinerGrammar = {}
MaxTimeout = 60*60*24 # 2 day
MaxParseTimeout = 60*5
CHECK = {'calculator', 'parson', 'tinyc', 'csv', 'mjs', 'mini-lisp', 'yxml', 'mini-c'}

### Check Recall

How many of the *valid* inputs from the golden grammar can be recognized by a parser using our grammar?

In [91]:
from fuzzingbook.Parser import IterativeEarleyParser

In [92]:
def check_recall(golden_grammar, my_grammar, maximum=Max_Recall, start_symbol='<START>', log=False):
    my_count = maximum
    ie = IterativeEarleyParser(my_grammar, start_symbol=start_symbol)
    golden = GrammarFuzzer(golden_grammar, start_symbol=start_symbol)
    success = 0
    while my_count != 0:
        src = golden.fuzz()
        try:
            my_count -= 1
            try:
                # print('?', repr(src), file=sys.stderr)
                for tree in ie.parse(src):
                    success += 1
                    break
                if log: print(maximum - my_count, '+', repr(src), success, file=sys.stderr)
            except:
                #print("Error:", sys.exc_info()[0], file=sys.stderr)
                if log: print(maximum - my_count, '-', repr(src), file=sys.stderr)
                pass
        except:
            pass
    return (success, maximum)

### Check Precision
How many of the inputs produced using our grammar are valid? (Accepted by the program).

In [93]:
def check_precision(name, grammar, maximum=Max_Precision, start_symbol='<START>', log=False):
    success = 0
    with ExpectError():
        fuzzer = GrammarFuzzer(grammar, start_symbol)
        for i in range(maximum):
            v = fuzzer.fuzz(key=start_symbol)
            c = check_it(v, name)
            success += (1 if c else 0)
            if log: print(i, repr(v), c)
    return (success, maximum)

In [94]:
class timeit():
    def __enter__(self):
        self.tic = datetime.now()
        return self
    def __exit__(self, *args, **kwargs):
        self.delta = datetime.now() - self.tic
        self.runtime = (self.delta.microseconds, self.delta)

### Timer

In [95]:
from datetime import datetime
from fuzzingbook.GrammarFuzzer import GrammarFuzzer

## Subjects

In [96]:
BMiner_p = {}
BMiner_r = {}

BMiner_t ={}

### Calculator

####  Golden Grammar

In [97]:
calc_golden = {
  "<START>": [
    "<expr>"
  ],
  "<expr>": [
    "<term>+<expr>",
    "<term>-<expr>",
    "<term>"
  ],
  "<term>": [
    "<factor>*<term>",
    "<factor>/<term>",
    "<factor>"
  ],
  "<factor>": [
    "(<expr>)",
    "<number>"
  ],
  "<number>": [
    "<integer>.<integer>",
    "<integer>"
  ],
  "<integer>": [
    "<digit><integer>",
    "<digit>"
  ],
  "<digit>": [ "0", "1", "2", "3", "4", "5", "6", "7", "8", "9" ]
}

#### Samples

In [98]:
calc_samples=[i.strip() for i in '''\
(1+2)*3/(423-334+9983)-5-((6)-(701))
(123+133*(12-3)/9+8)+33
(100)
21*3
33/44+2
100
23*234*22*4
1+2
31/20-2
555+(234-445)
1-(41/2)
443-334+33-222
'''.split('\n') if i.strip()]

In [99]:
calc_grammar = binary_miner([calc_samples[0]], 'subject/calc/calc_parse.c')
calc_grammar

(1+2)*3/(423-334+9983)-5-((6)-(701))
<main> ['<parse_expr-0-c>']
<parse_expr-2> ['<parse_num-0-c>', '<parse_expr-3>']
<parse_expr-5> ['<parse_expr-0-c>', '<parse_expr-6>']
<parse_expr-8> ['<strlen-0-c>', '<parse_expr-9>']
<__GI__dl_addr-1> ['<_dl_find_dso_for_object-1>', '<__tunable_get_val-0-c>']
<parse_expr-6> ['<parse_expr-6-s>', '<parse_expr-0-c>']
<parse_expr-3> ['<parse_expr-3-s>', '<parse_num-0-c>']
<_int_malloc-1> ['0']
<parse_num-2> ['<malloc-0-c>', '<__ctype_b_loc-0-c>']
<parse_expr-11> ['<__ctype_b_loc-0-c>', '<parse_expr-12>']
<strlen-10> ['7']
<parse_expr-12> ['<parse_num-0-c>', '<parse_expr-13>']
<__tunable_get_val-2> ['<parse_expr-0-c>', '<__tunable_get_val-3>']
<strlen-9> ['6']
<strlen-3> ['*']
<strlen-8> ['5']
<parse_expr-13> ['<strlen-0-c>', '<parse_expr-14>']
<__tunable_get_val-3> ['<__tunable_get_val-3-s>', '<__tunable_get_val-4>']
<__tunable_get_val-4> ['<parse_num-0-c>', '<__tunable_get_val-5>']
<parse_expr-14> ['<__ctype_b_loc-0-c>', '<parse_num-0-c>']
<__tunable

{'<START>': ['<parse_expr-0-c>'],
 '<parse_expr-0-c>': ['<malloc-0-c>',
  '<parse_expr-0-c><parse_expr-6-s><parse_expr-0-c>',
  '<parse_num-0-c><parse_expr-3-s><parse_num-0-c>',
  '<strlen-0-c><parse_expr-9>'],
 '<malloc-0-c>': ['<_dl_find_dso_for_object-1><__tunable_get_val-0-c>',
  '<_int_malloc-0-c>'],
 '<parse_expr-6-s>': ['<strlen-0-c>', '<strlen-0-c><parse_expr-6-s>'],
 '<parse_num-0-c>': ['<malloc-0-c><__ctype_b_loc-0-c>', '<parse_num-1-s>'],
 '<parse_expr-3-s>': ['<strlen-0-c>', '<strlen-0-c><parse_expr-3-s>'],
 '<strlen-0-c>': ['*',
  '5',
  '6',
  '7',
  '<_dl_find_dso_for_object-1>',
  '<strlen-11>',
  '<strlen-2>',
  '<strlen-4>',
  '<strlen-5>',
  '<strlen-6>',
  '<strlen-7>'],
 '<parse_expr-9>': ['<parse_num-0-c>', '<parse_num-0-c><parse_expr-10>'],
 '<_dl_find_dso_for_object-1>': ['('],
 '<__tunable_get_val-0-c>': ['<__tunable_get_val-1>', '<_int_malloc-0-c>'],
 '<_int_malloc-0-c>': ['0', '<__ctype_b_loc-6>'],
 '<__tunable_get_val-1>': ['<__tunable_get_val-0-c>',
  '<__t

In [102]:
f = GrammarFuzzer(calc_grammar, start_symbol='<START>')
f.fuzz()

'(00*+*068580'

In [ ]:
if 'calculator' in CHECK:
    result = check_recall(calc_golden, calc_grammar, start_symbol='<START>')
    BMiner_r['calculator.py'] = result
    print(result)

### Parson

Parson is a lightweight json library written in C.

#### Samples

In [ ]:
json_samples = [i.strip() for i in '''\
{"emptya":[],"emptyh":{},"emptystr":"","null":"null"}
{"color":"blue","category":"hue","type":"primary","code":{"rgba":[0,0,255,1],"hex":"#00F"}}
{"color":"yellow","category":"hue","wetype":"primary","code":{"rgba":[255,255,0,1],"hex":"#FF0"}}
{"color":"green","category":"hue","type":"secondary","code":{"rgba":[0,255,0,1],"hex":"#0F0"}
'''.split('\n') if i.strip()]

In [ ]:
#parson_grammar = binary_miner([json_samples[0]], 'subject/json/cJSON.c')
#parson_grammar

### MJS

#### Golden Grammar

In [96]:
import string
mjs_golden = {
    '<START>': ['<program>'],
    '<program>': ['<statement>'],
    '<statement>': ['if <paren_expr> <statement>', 'if <paren_expr> <statement> else <statement>',
                   'while <paren_expr> <statement>', 'do <statement> while <paren_expr>;',
                   '{ <statement> }', '<expr> ;', ';'],
    '<paren_expr>': ['(<expr>)'],
    '<expr>': ['<test>', '<id>=<expr>'],
    '<test>': ['<sum>', '<sum><<sum>'],
    '<sum>': ['<term>', '<sum>+<term>', '<sum>-<term>'],
    '<term>': ['<id>', '<int>', '<paren_expr>'],
    '<id>': list(string.ascii_lowercase),
    '<int>': list(string.digits)
}

#### Samples

In [97]:
mjs_samples=[i.strip() for i in '''\
{ i=7; if (i<5) x=1; if (i<10) y=2; }
{ i=1; while (i<100) i=i+i; }
a=b=c=2<3;
{ i=1; while ((i=i+10)<50) ; }
'''.split('\n') if i.strip()]

In [295]:
mjs_grammar = binary_miner(mjs_samples, 'subject/mjs/mjs.c')
mjs_grammar

{ i=7; if (i<5) x=1; if (i<10) y=2; }
<main> ['<parse_statement_list-1>']
<parse_if-1> ['<pnext-0-c>', '<parse_if-2>']
<parse_comparison-1> ['<parse_shifts-1>', '<pnext-0-c>']
<parse_if-2> ['<parse_assignment-0-c>', '<parse_if-3>']
<parse_if-3> ['<pnext-0-c>', '<parse_block_or_stmt-1>']
<findtok-1> ['0']
<parse_block_or_stmt-1> ['<pnext-0-c>', '<parse_statement-0-c>']
<longtok3-1> ['1']
<longtok3-2> ['2']
<*ABS*+0xa27b0-2> [';']
<longtok3-3> ['5']
<*ABS*+0xa27b0-1> [')']
<skip_spaces_and_comments-3> ['{']
<strlen-1> ['7']
<mjs_is_space-4> ['x']
<mjs_is_space-2> ['(']
<mjs_is_space-5> ['y']
<mjs_is_space-6> ['}']
<mjs_is_alpha-1> ['<']
<mjs_is_alpha-2> ['=']
<mjs_is_alpha-3> ['f']


{'<START>': ['<parse_statement_list-1>'],
 '<parse_statement_list-1>': ['<parse_statement_list-1-s>',
  '<parse_statement_list-1-s><pnext-0-c>'],
 '<parse_statement_list-1-s>': ['<pnext-0-c><parse_statement-0-c>',
  '<pnext-0-c><parse_statement-0-c><parse_statement_list-1-s>'],
 '<pnext-0-c>': ['<*ABS*+0xa27b0-0-c>', '<longtok3-0-c>', '<pnext-3>'],
 '<parse_statement-0-c>': ['<parse_assignment-0-c>',
  '<parse_statement_list-1>',
  '<pnext-0-c><parse_assignment-0-c><pnext-0-c><pnext-0-c><parse_statement-0-c>'],
 '<parse_assignment-0-c>': ['<parse_shifts-1><pnext-0-c>', '<pnext-0-c>'],
 '<parse_shifts-1>': ['<pnext-0-c>', '<pnext-0-c>0'],
 '<*ABS*+0xa27b0-0-c>': [')', ';'],
 '<longtok3-0-c>': ['1', '2', '5'],
 '<pnext-3>': ['<skip_spaces_and_comments-0-c>',
  '<skip_spaces_and_comments-0-c><pnext-4>'],
 '<skip_spaces_and_comments-0-c>': ['<mjs_is_space-0-c>',
  '<skip_spaces_and_comments-2>',
  '{'],
 '<pnext-4>': ['<mjs_is_ident-0-c>', '<mjs_is_ident-0-c>7'],
 '<mjs_is_space-0-c>': ['(

In [ ]:
if 'mjs' in CHECK:
    result = check_recall(mjs_golden, mjs_grammar, start_symbol='<START>')
    BMiner_r['mjs.c'] = result
    print(result)

In [301]:
f = GrammarFuzzer(mjs_grammar, start_symbol='<START>')
f.fuzz()

'{;5))21;;55;5;'

### Duktape
Duktape is an embeddable Javascript engine, with a focus on portability and compact footprint.

In [ ]:
duktape_samples=[i.strip() for i in '''\
"a = 'Hello world!';"
'''.split('\n') if i.strip()]

In [ ]:
duktape_grammar = binary_miner(duktape_samples, 'subject/duktape/duktape.c')
duktape_grammar

### CSV 

#### The Golden Grammar

In [302]:
import string

CSV_GRAMMAR = {
    '<START>': ['<csvline>'],
    '<csvline>': ['<items>'],
    '<items>': ['<item>,<items>', '<item>'],
    '<item>': ['<letters>'],
    '<letters>': ['<letter><letters>', '<letter>'],
    '<letter>': list(string.ascii_letters + string.digits + string.punctuation + ' \t\n')
}

#### Samples

In [303]:
from fuzzingbook.GrammarMiner import VEHICLES

In [304]:
csv_grammar = binary_miner(VEHICLES, 'subject/csv/csv.c')
csv_grammar

1997,van,Ford,E350
2000,car,Mercury,Cougar
1999,car,Chevy,Venture
<main> ['<__GI__dl_addr-1>']
<__GI__dl_addr-1> ['<_dl_find_dso_for_object-0-c>', '<__tunable_get_val-0-c>']
<_dl_find_dso_for_object-1> ['1']
<_dl_find_dso_for_object-2> ['2']
<csv_load-1> ['<append_row-1>', '<csv_load-2-s>']
<append_row-1> ['<realloc-0-c>', '<strcpy-1>']
<read_next_field-2> ['<add_char-1>', '<read_next_field-3-s>']
<getc-14> ['d']
<getc-22> ['u']
<getc-11> ['V']
<getc-4> ['5']
<getc-5> ['7']
<getc-19> ['o']
<getc-9> ['F']
<getc-18> ['n']
<getc-20> ['r']
<getc-3> ['3']
<getc-15> ['e']
<getc-8> ['E']
<getc-21> ['t']
<getc-10> ['M']
<getc-12> ['a']
<getc-23> ['v']
<getc-16> ['g']
<getc-17> ['h']
<getc-13> ['c']
<getc-24> ['y']
<getc-7> ['C']


{'<START>': ['<_dl_find_dso_for_object-0-c><__tunable_get_val-0-c>'],
 '<_dl_find_dso_for_object-0-c>': ['1', '2'],
 '<__tunable_get_val-0-c>': ['<__tunable_get_val-1>',
  '<_int_malloc-1>',
  '<csv_load-0-c>'],
 '<__tunable_get_val-1>': ['<__tunable_get_val-0-c>',
  '<__tunable_get_val-0-c><csv_load-0-c>'],
 '<_int_malloc-1>': ['0'],
 '<csv_load-0-c>': ['<read_next_field-1>',
  '<realloc-0-c><strcpy-1><csv_load-2-s>'],
 '<read_next_field-1>': ['<read_next_field-1-s>',
  '<read_next_field-1-s><add_char-1><read_next_field-3-s>'],
 '<realloc-0-c>': ['<_int_malloc-1>', '<_int_realloc-1>'],
 '<strcpy-1>': [','],
 '<csv_load-2-s>': ['<read_next_field-1>',
  '<read_next_field-1><csv_load-2-s>'],
 '<read_next_field-1-s>': ['<getc-0-c>', '<getc-0-c><read_next_field-1-s>'],
 '<add_char-1>': ['9', '9<realloc-0-c>'],
 '<read_next_field-3-s>': ['<getc-0-c>', '<getc-0-c><read_next_field-3-s>'],
 '<getc-0-c>': ['3',
  '5',
  '7',
  '<_int_malloc-1>',
  '<_int_realloc-1>',
  '<strcpy-1>',
  'C',
  'E

In [ ]:
if 'csv' in CHECK:
    result = check_recall(CSV_GRAMMAR, csv_grammar, start_symbol='<START>')
    BMiner_r['csv.c'] = result
    print(result)

In [312]:
f = GrammarFuzzer(csv_grammar, start_symbol='<START>')
f.fuzz()

'2t970,oh'

### TinyC

#### Golden Grammar

In [313]:
import string
tinyc_golden = {
    '<START>': ['<program>'],
    '<program>': ['<statement>'],
    '<statement>': ['if <paren_expr> <statement>', 'if <paren_expr> <statement> else <statement>',
                   'while <paren_expr> <statement>', 'do <statement> while <paren_expr>;',
                   '{ <statement> }', '<expr> ;', ';'],
    '<paren_expr>': ['(<expr>)'],
    '<expr>': ['<test>', '<id>=<expr>'],
    '<test>': ['<sum>', '<sum><<sum>'],
    '<sum>': ['<term>', '<sum>+<term>', '<sum>-<term>'],
    '<term>': ['<id>', '<int>', '<paren_expr>'],
    '<id>': list(string.ascii_lowercase),
    '<int>': list(string.digits)
}

#### Samples

In [314]:
tiny_samples=[i.strip() for i in '''\
{ i=1; while (i<100) i=i+i; }
a=b=c=2<3;
{ i=125; j=100; while (i-j) if (i<j) j=j-i; else i=i-j; }
{ i=1; do i=i+10; while (i<50); }
{ i=1; while ((i=i+10)<50) ; }
{ i=7; if (i<5) x=1; if (i<10) y=2; }
'''.split('\n') if i.strip()]

In [315]:
tiny_grammar = binary_miner([tiny_samples[0]], 'subject/tinyC/tiny.c')
tiny_grammar

{ i=1; while (i<100) i=i+i; }
<main> ['<new_node-0-c>']
<new_node-1> ['(']
<__GI__dl_addr-1> ['<_dl_find_dso_for_object-1>', '<__tunable_get_val-0-c>']
<_dl_find_dso_for_object-1> ['{']
<_int_malloc-2> ['1']
<__tunable_get_val-2> ['<next_sym-1>', '<statement-0-c>']
<statement-3> ['<next_sym-1>', '<statement-3-c>']
<next_ch-11> ['l']
<next_ch-6> ['<']
<next_ch-2> [')']
<next_ch-12> ['w']
<next_ch-5> [';']
<next_ch-8> ['e']
<next_ch-3> ['+']
<statement-4> ['<new_node-0-c>', '<statement-5>']
<next_ch-7> ['=']
<next_ch-9> ['h']
<next_ch-1> [' ']
<statement-8> ['<paren_expr-1>', '<statement-0-c>']
<next_ch-13> ['}']
<paren_expr-1> ['<next_sym-1>', '<statement-1>']
<statement-5> ['<statement-0-c>', '<statement-6>']
<statement-6> ['<new_node-0-c>', '<statement-0-c>']
<expr-1> ['<next_sym-1>', '<expr-0-c>']
<test-2> ['<new_node-0-c>', '<test-3>']
<test-3> ['<next_sym-1>', '<sum-0-c>']
<sum-1> ['<next_sym-1>', '<term-0-c>']
<__tunable_get_val-0-c> ['<next_sym-1>', '<statement-0-c>']


{'<START>': ['<new_node-0-c>'],
 '<new_node-0-c>': ['(', '<malloc-0-c>'],
 '<malloc-0-c>': ['<_int_malloc-0-c>', '{<next_sym-1><statement-0-c>'],
 '<_int_malloc-0-c>': ['1', '<next_ch-4>'],
 '<next_sym-1>': ['<next_sym-1-s>', '<next_sym-1-s><strcmp-1>'],
 '<statement-0-c>': ['<next_sym-1><statement-3-c>', '<statement-1>'],
 '<next_ch-4>': ['0'],
 '<next_sym-1-s>': ['<next_ch-0-c>', '<next_ch-0-c><next_sym-1-s>'],
 '<strcmp-1>': ['i'],
 '<next_ch-0-c>': [' ',
  ')',
  '+',
  ';',
  '<',
  '<next_ch-4>',
  '<strcmp-1>',
  '=',
  'e',
  'h',
  'l',
  'w',
  '}'],
 '<statement-3-c>': ['<new_node-0-c><statement-0-c><new_node-0-c><statement-0-c>',
  '<next_sym-1><statement-1><statement-0-c>'],
 '<statement-1>': ['<expr-0-c><next_sym-1>'],
 '<expr-0-c>': ['<next_sym-1><expr-0-c>', '<test-1>'],
 '<test-1>': ['<sum-0-c>', '<sum-0-c><new_node-0-c><next_sym-1><sum-0-c>'],
 '<sum-0-c>': ['<next_sym-1><term-0-c>', '<term-0-c>'],
 '<term-0-c>': ['<new_node-0-c>', '<next_sym-1>']}

In [ ]:
if 'tinyc' in CHECK:
    result = check_recall(calc_golden, tiny_grammar, start_symbol='<START>')
    BMiner_r['tiny.c'] = result
    print(result)

In [335]:
f = GrammarFuzzer(tiny_grammar, start_symbol='<START>')
f.fuzz()

'{<e)}ll1(; lhi'

### Yxml

#### The Golden Grammar

In [336]:
from fuzzingbook.Grammars import srange
import string

In [337]:
XML_GRAMMAR = {
    "<START>": ["<xml-tree>"],
    "<xml-tree>": ["<text>",
                   "<xml-open-tag><xml-tree><xml-close-tag>", 
                   "<xml-openclose-tag>", 
                   "<xml-tree><xml-tree>"],
    "<xml-open-tag>":      ["<<id>>", "<<id> <xml-attribute>>"],
    "<xml-openclose-tag>": ["<<id>/>", "<<id> <xml-attribute>/>"],
    "<xml-close-tag>":     ["</<id>>"],
    "<xml-attribute>" :    ["<id>=<id>", "<xml-attribute> <xml-attribute>"],
    "<id>":                ["<letter>", "<id><letter>"],
    "<text>" :             ["<text><letter_space>","<letter_space>"],
    "<letter>":            srange(string.ascii_letters + string.digits +"\""+"'"+"."),
    "<letter_space>":      srange(string.ascii_letters + string.digits +"\""+"'"+" "+"\t"),
}

#### Samples

In [338]:
xml_samples=[i.strip() for i in '''\
<note></note>
<to>Tove</to>
<from>Jani</from>
<heading>Reminder</heading>
'''.split('\n') if i.strip()]

In [339]:
yxml_grammar = binary_miner([xml_samples[0]], 'subject/xml/yxml.c')
yxml_grammar

<note></note>
<main> ['<__GI__dl_addr-1>']
<__GI__dl_addr-1> ['<_dl_find_dso_for_object-1>', '<__tunable_get_val-0-c>']
<__tunable_get_val-3> ['<_int_malloc-0-c>', '<_int_malloc-0-c>']
<yxml_parse-1> ['/']


{'<START>': ['<_dl_find_dso_for_object-1><__tunable_get_val-0-c>'],
 '<_dl_find_dso_for_object-1>': ['<'],
 '<__tunable_get_val-0-c>': ['<__tunable_get_val-1>',
  '<_int_malloc-0-c><_int_malloc-0-c>'],
 '<__tunable_get_val-1>': ['<__tunable_get_val-0-c>',
  '<__tunable_get_val-0-c><__tunable_get_val-2-s>'],
 '<_int_malloc-0-c>': ['<yxml_elemclose-3>', '<yxml_parse-5>'],
 '<__tunable_get_val-2-s>': ['<yxml_parse-0-c>',
  '<yxml_parse-0-c><__tunable_get_val-2-s>'],
 '<yxml_parse-0-c>': ['/',
  '<_dl_find_dso_for_object-1>',
  '<yxml_elemclose-0-c>',
  '<yxml_parse-5>',
  '<yxml_pushstackc-0-c>'],
 '<yxml_elemclose-0-c>': ['<yxml_elemclose-1>',
  '<yxml_elemclose-2>',
  '<yxml_elemclose-3>',
  '<yxml_elemclose-4>'],
 '<yxml_parse-5>': ['n'],
 '<yxml_pushstackc-0-c>': ['<yxml_elemclose-1>',
  '<yxml_elemclose-2>',
  '<yxml_elemclose-4>'],
 '<yxml_elemclose-1>': ['>'],
 '<yxml_elemclose-2>': ['e'],
 '<yxml_elemclose-3>': ['o'],
 '<yxml_elemclose-4>': ['t']}

In [ ]:
if 'yxml' in CHECK:
    result = check_recall(XML_GRAMMAR, yxml_grammar, start_symbol='<START>')
    BMiner_r['yxml.c'] = result
    print(result)

In [347]:
f = GrammarFuzzer(yxml_grammar, start_symbol='<START>')
for i in range(100):
    print(f.fuzz())

<oo
<no
<non/>
<oo<<
<no////>><////
<nn
<on
<no
<oo
<nne
<on
<onn/</ont
<no
<on
<non
<no
<nn
<no
<no
<not/ten<<
<nn
<on
<no
<no
<oo
<no
<oo/
<on
<on
<one/e/
<on<nt
<no<n
<nnn
<on
<no
<on
<no
<nnt<
<oo>
<nn/et>
<on
<on
<oo<t/n
<on
<oon
<nn
<oo
<onent
<oo
<oo
<ootoe
<oo
<oo
<onon
<no
<oo
<onn
<oo
<nn
<oo
<no
<on
<nn
<on
<no
<nn
<no
<oo<<<
<on/n
<on
<oo
<no
<oo>tnt
<no
<onnnt<
<on
<no
<nn/
<oo
<oneon
<oot<en/>/<
<non
<no
<oo
<nn
<on/<
<nn>
<nnn
<nn
<oo
<noet>e/
<no
<oo
<on
<on/>
<no
<on
<on
<no
<oo/


## Results